# 라이브러리

In [150]:
import numpy as np
import pandas as pd
import pickle
import os
import geocoder
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = "/Library/Fonts/Arial Unicode.ttf"
fontprop = fm.FontProperties(fname=font_path, size=18)

# 데이터 불러오기

In [159]:
path = "/Users/heogeonhyeock/Desktop/인빅_프로젝트/교통량"
file_list = os.listdir(path)

for i in range(len(file_list)):
    try:
        globals()["trafic_{}".format(i)] = pd.read_csv(path + "/" + file_list[i],
                                                        encoding='cp949')
    except:
        pass
trafic = pd.concat([trafic_1,trafic_2,trafic_3,trafic_4])
trafic

,집계일자,출발권역명,수도권도착,강원도착,대전충남도착,광주전남도착,대구경북도착,부산경남도착,전북도착,충북도착,Unnamed: 10
0,20210401,수도권본부,537405,56887,72633,3576,8589,6016,7166,52410,NaN
1,20210401,강원본부,57238,97204,1966,511,2067,877,668,15185,NaN
2,20210401,대전충남본부,71187,2196,143868,4520,9162,4823,28094,16653,NaN
3,20210401,광주전남본부,3259,533,4109,173834,4057,14154,20971,1390,NaN
4,20210401,대구경북본부,8044,1978,8412,3942,260308,39287,2250,8262,NaN
...,...,...,...,...,...,...,...,...,...,...,...
243,20210331,광주전남본부,2933,464,3974,173260,3948,12327,20827,1392,NaN
244,20210331,대구경북본부,7577,1814,8260,3847,233399,37800,2213,8039,NaN
245,20210331,부산경남본부,5595,840,3599,12096,34302,455447,3758,1810,NaN
246,20210331,전북본부,6759,598,27586,20706,2080,4033,33833,2691,NaN


In [152]:
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/FC주문/FC_order_distance.pickle","rb") as fr:
    FC_order = pickle.load(fr).reset_index(drop=True)
FC_order["송화 -> 터미널"] = None
FC_order["터미널 -> 수화"] = None
FC_order = FC_order[["INS_DATE","SHPR_ADDR_1","CNEE_ADDR_1","송화 -> 터미널","터미널 -> 수화"]]
FC_order

,INS_DATE,SHPR_ADDR_1,CNEE_ADDR_1,송화 -> 터미널,터미널 -> 수화
0,20210303,경기도,경상남도,None,None
1,20210303,경기도,경상남도,None,None
2,20210303,경기도,서울특별시,None,None
3,20210303,경기도,경기도,None,None
4,20210303,경기도,경기도,None,None
...,...,...,...,...,...
1585442,20210702,경기도,서울특별시,None,None
1585443,20210702,경기도,경기도,None,None
1585444,20210703,경기도,제주특별자치도,None,None
1585445,20210704,경기도,대전광역시,None,None


# 지역 구분

In [153]:
loc_dict = {"수도권본부" : ["경기도", "서울특별시", "경기도", "인천광역시", "서울", "경기", "인천", "인천시"],
     "강원본부" : ["강원도", "강원"],
     "대전충남본부" : ["충청남도", "대전광역시", "세종특별자치시", "대전", "충남"],
     "광주전남본부" : ["전라남도", "광주광역시", "광주", "전남"],
     "대구경북본부" : ["경상북도", "대구광역시", "경북", "대구", "대구시"],
     "부산경남본부" : ["경상남도", "부산광역시", "울산광역시", "경남", "부산", "울산", "부산시"],
     "전북본부" : ["전라북도", "전북"],
     "충북본부" : ["충청북도", "충북"],
     "권역 외" : ["제주특별자치도", "제주"]}

In [224]:
pd.options.mode.chained_assignment = None  # default='warn'

FC_time = np.array(FC_order["INS_DATE"])

for index in FC_order.index[:-10:-1]:
    for i in list(loc_dict.keys())[:-1]:
        if FC_order["SHPR_ADDR_1"][index] in loc_dict.get(i):
            FC_order["송화 -> 터미널"][index] = trafic[(trafic["집계일자"] == FC_time[index]) &\
                                                   (trafic["출발권역명"] == i)]["수도권도착"]
        if FC_order["CNEE_ADDR_1"][index] in loc_dict.get(i):
            FC_order["터미널 -> 수화"][index] = int(trafic[(trafic["집계일자"] == FC_time[index]+1) &\
                                                   (trafic["출발권역명"] == "수도권본부")][i[:-2]+"도착"])

ValueError: No axis named 1 for object type Series

In [220]:
a = [1,2,3,4,5,6,7,8,9]

In [223]:
a[:-4:-1]

[9, 8, 7]

In [168]:
FC_order.head(10)

,INS_DATE,SHPR_ADDR_1,CNEE_ADDR_1,송화 -> 터미널,터미널 -> 수화
0,20210303,경기도,경상남도,514547.0,None
1,20210303,경기도,경상남도,514547.0,None
2,20210303,경기도,서울특별시,514547.0,None
3,20210303,경기도,경기도,514547.0,None
4,20210303,경기도,경기도,514547.0,None
5,20210303,경기도,경기도,514547.0,None
6,20210303,경기도,경상남도,514547.0,None
7,20210303,경기도,서울특별시,514547.0,None
8,20210303,경기도,경기도,514547.0,None
9,20210303,경기도,대전광역시,514547.0,None


In [118]:
trafic[(trafic["집계일자"] == FC_order["INS_DATE"][1]) & (trafic["출발권역명"] == "수도권본부")]

,집계일자,출발권역명,수도권도착,강원도착,대전충남도착,광주전남도착,대구경북도착,부산경남도착,전북도착,충북도착,Unnamed: 10
16,20210303,수도권본부,514547,52697,69022,3025,8135,5729,6400,47782,NaN


In [125]:
loc_dict.get("수도권본부")

['경기도', '서울특별시', '경기도', '인천광역시', '서울', '경기', '인천', '인천시']

In [111]:
trafic["출발권역명"] == "수도권본부"

0       True
1      False
2      False
3      False
4      False
       ...  
243    False
244    False
245    False
246    False
247    False
Name: 출발권역명, Length: 976, dtype: bool

,집계일자,출발권역명,수도권도착,강원도착,대전충남도착,광주전남도착,대구경북도착,부산경남도착,전북도착,충북도착,Unnamed: 10
16,20210303,수도권본부,514547,52697,69022,3025,8135,5729,6400,47782,NaN


In [101]:
trafic[trafic["출발권역명"] == "수도권본부", (trafic["집계일자"] == FC_order["INS_DATE"][1])]

TypeError: '(0       True
1      False
2      False
3      False
4      False
       ...  
243    False
244    False
245    False
246    False
247    False
Name: 출발권역명, Length: 976, dtype: bool, 0      False
1      False
2      False
3      False
4      False
       ...  
243    False
244    False
245    False
246    False
247    False
Name: 집계일자, Length: 976, dtype: bool)' is an invalid key

수도권본부 : 경기도, 서울특별시, 경기도, 인천광역시, 서울, 경기, 인천, 인천시
강원본부 : 강원도, 강원
대전충남본부 : 충청남도, 대전광역시, 세종특별자치시, 대전, 충남
광주전남본부 : 전라남도, 광주광역시, 광주, 전남
대구경북본부 : 경상북도, 대구광역시, 경북, 대구, 대구시
부산경남본부 : 경상남도, 부산광역시, 울산광역시, 경남, 부산, 울산, 부산시
전북본부 : 전라북도, 전북
충북본부 : 충청북도, 충북
권역 외 : 제주특별자치도, 제주

In [25]:
find_key(loc_dict, "강원도")

StopIteration: 